In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%load_ext autotime

# Sample Submission

In [2]:
#sample = pd.read_csv("sample_submission.csv")
#sample[:3]

time: 237 µs


In [3]:
#sample.prediction.value_counts()

time: 242 µs


# Test

In [4]:
#test_orig = pd.read_csv("test.csv")
#test_orig[:3]

time: 217 µs


# Train Cleaning

In [5]:
train_orig = pd.read_csv("/home/sroberts/train.csv", engine='python')

time: 56.8 s


In [6]:
#y_train = train_orig['target'].copy()
#x_train = train_orig.loc[:, train_orig.columns != 'target'].copy()
#x_train = train_orig.loc[:, ~train_orig.columns.isin(['target','severe_toxicity','obscene',
#                                                    'identity_attack','insult','threat'])].copy()

time: 399 µs


In [7]:
train_orig[:2]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.0,"This is so cool. It's like, 'would you want yo...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.0,Thank you!! This would make my life a lot less...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4


time: 44.6 ms


In [8]:
# toxic comments
train_orig[train_orig.target >= 0.5][:2]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.000000,4,47
5,59859,0.666667,ur a sh*tty comment.,0.047619,0.638095,0.000000,0.333333,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:50.865549+00,2,NaN,2006,rejected,0,0,0,0,0,0.009524,0,105


time: 103 ms


In [9]:
#x_train[:2]

time: 270 µs


# Fill NaNs with Zeros

In [10]:
#x_train.fillna(0, inplace=True)

time: 8.62 ms


In [11]:
train_orig.fillna(0, inplace=True)

time: 787 ms


# describe/summarize

In [12]:
desc_df = train_orig.copy()
desc_df.loc[desc_df.target >= 0.5, "toxic"] = True
desc_df.toxic.fillna(False,inplace=True)

time: 1.57 s


In [13]:
rel_col = list(set(desc_df.columns.tolist()) - set(['target','id']))
desc_df[rel_col].groupby("toxic").describe()

disagree                                                sexual_explicit                                              heterosexual                                                  jewish                                              psychiatric_or_mental_illness                                                  muslim                                                   likes                                                     asian                                               parent_id                                                                            latino                                               christian                                                   funny                                                      male                                                   black                                              other_disability                                              identity_attack                                                          buddhist  \
           count      mean       std  min  25%  50%  75%    max           count      mean       std  min  25%  50%  75%  max        count      mean       std  min  25%  50%  75%  max      count      mean       std  min  25%  50%  75%  max                         count      mean       std  min  25%  50%  75%  max      count      mean       std  min  25%  50%  75%  max      count      mean       std  min  25%  50%  75%    max      count      mean       std  min  25%  50%  75%  max      count          mean           std  min  25%       50%         75%        max      count      mean       std  min  25%  50%  75%  max      count      mean       std  min  25%  50%  75%  max      count      mean       std  min  25%  50%  75%    max      count      mean       std  min  25%  50%  75%  max      count      mean       std  min  25%  50%  75%  max            count      mean       std  min  25%  50%  75%  max           count      mean       std  min  25%       50%       75%  max      count   
toxic                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
False  1660540.0  0.578480  1.854387  0.0  0.0  0.0  0.0  187.0       1660540.0  0.003806  0.028521  0.0  0.0  0.0  0.0  1.0    1660540.0  0.000618  0.020063  0.0  0.0  0.0  0.0  1.0  1660540.0  0.003652  0.055521  0.0  0.0  0.0  0.0  1.0                     1660540.0  0.002314  0.039499  0.0  0.0  0.0  0.0  1.0  1660540.0  0.009336  0.089722  0.0  0.0  0.0  0.0  1.0  1660540.0  2.437559  4.695471  0.0  0.0  1.0  3.0  300.0  1660540.0  0.002576  0.040615  0.0  0.0  0.0  0.0  1.0  1660540.0  2.128586e+06  2.612411e+06  0.0  0.0  526010.0  5360377.25  6333965.0  1660540.0  0.001231  0.026288  0.0  0.0  0.0  0.0  1.0  1660540.0  0.021084  0.126729  0.0  0.0  0.0  0.0  1.0  1660540.0  0.280510  1.065251  0.0  0.0  0.0  0.0  102.0  1660540.0  0.022545  0.129692  0.0  0.0  0.0  0.0  1.0  1660540.0  0.005761  0.069089  0.0  0.0  0.0  0.0  1.0        1660540.0  0.000260  0.006527  0.0  0.0  0.0  0.0  0.6       1660540.0  0.014580  0.054317  0.0  0.0  0.000000  0.000000  0.8  1660540.0   
True    144334.0  0.652112  

time: 6.57 s


# lower case, stopwords, puncutation

In [14]:
#import nltk
#nltk.download('all')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')

time: 298 µs


In [15]:
pd.options.display.max_colwidth = 1000
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
import string

time: 518 ms


In [16]:
#import nltk
#nltk.download('stopwords')

english_stopwords = set(stopwords.words('english'))
w_tokenizer = WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

time: 2.2 ms


In [17]:
def clean_message(msg):
    msg = msg.lower()
    msg_tokens = nltk.word_tokenize(msg)
    clean_msg_tokens = [w for w in msg_tokens if w not in english_stopwords]
    clean_msg_tokens_puct = [w for w in clean_msg_tokens if w not in string.punctuation]
    lemmatized_token = [lemmatizer.lemmatize(w) for w in clean_msg_tokens_puct]
    return lemmatized_token

time: 7.12 ms


In [18]:
# poorn
#train_orig['clean'] = train_orig.comment_text.apply(clean_message)

time: 8.23 ms


In [19]:
#nltk.download('wordnet')


time: 8.07 ms


In [20]:
train_orig["clean"] = train_orig.comment_text.apply(lambda x: " ".join([words.strip(string.punctuation).lower()
for words in [word for word in x.split() if (word not in string.punctuation) & (word not in english_stopwords)]]))
train_orig["clean"] = train_orig["clean"].apply(lambda x: " ".join([lemmatizer.lemmatize(w) 
                                                                    for w in w_tokenizer.tokenize(x)]))

time: 4min 32s


In [21]:
train_orig['clean'][:2]

0          this cool it's like would want mother read this really great idea well done
1    thank you this would make life lot le anxiety-inducing keep up let anyone get way
Name: clean, dtype: object

time: 4.25 ms


# vectorize

In [22]:
from collections import Counter
import numpy as np

time: 8.31 ms


In [23]:
# pooran
#voc_counter = Counter(messages_df.clean_message.sum())

time: 8.49 ms


In [24]:
voc_counter = Counter(" ".join(train_orig.clean).split(" "))

time: 16.3 s


In [25]:
#for w,c in voc_counter.items():
#    print(w,c)

time: 7.63 ms


In [26]:
voc_set = dict((k, v) for k, v in voc_counter.items() if v > 10)
voc_dict = {w:i for w,i in zip(voc_set, range(len(voc_set)))}

time: 151 ms


In [27]:
train_orig["clean"] = train_orig["clean"].apply(lambda x: x.split())
train_orig.loc[train_orig.target >= 0.5, "toxic"] = True
train_orig.toxic.fillna(False,inplace=True)

time: 11.4 s


In [28]:
#train_orig = train_orig.sample(500000)

time: 293 µs


In [29]:
X = np.zeros((len(train_orig), len(voc_set)), dtype = np.int8 )
for msg_idx, msg in enumerate(train_orig['clean']):
    for word in set(msg):
        if word in voc_dict:
            X[msg_idx][voc_dict[word]] = msg.count(word)
            
y = np.where(train_orig.toxic == True, 1, 0)

time: 1min 31s


# Save

In [30]:
X.shape

(1804874, 63574)

time: 1.87 ms


In [ ]:
# np.savetxt('y.txt', y, fmt='%d')
# np.savetxt('X.txt', X, fmt='%d')

# !!! restart kernel !!!

# Load

In [ ]:
# import numpy as np
# X = np.loadtxt('X.txt', dtype=int)
# y = np.loadtxt('y.txt', dtype=int)

# Model

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

time: 7.95 ms


In [32]:
from sklearn.linear_model import SGDClassifier

time: 8.64 ms


In [ ]:
#train_size=.8
#train_indicies = int(X.shape[0]*train_size)

#np.random.shuffle(X)
#train_X,test_X = X[:train_indicies,:],X[train_indicies:,:]
#train_y,test_y = y[:train_indicies],  y[train_indicies:]

In [33]:
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=.2,random_state=2)

time: 1min 14s


In [66]:
lr = SGDClassifier(loss="log", n_jobs=-1)

#classes = np.unique(["toxic", "not_toxic"])
classes = np.unique([0, 1])

time: 1.08 ms


In [67]:
classes

array([0, 1])

time: 3.03 ms


In [35]:
train_X.shape

(1443899, 63574)

time: 2.28 ms


In [56]:
train_y.shape

(1443899,)

time: 1.81 ms


In [68]:
minibatches = [(train_X[:100000,:],train_y[:100000]),
               (train_X[100001:200000,:],train_y[100001:200000]),
               (train_X[200001:300000,:],train_y[200001:300000]),
               (train_X[300001:400000,:],train_y[300001:400000]),
               (train_X[400001:500000,:],train_y[400001:500000]),
               (train_X[500001:600000,:],train_y[500001:600000]),
               (train_X[600001:700000,:],train_y[600001:700000]),
               (train_X[700001:800000,:],train_y[700001:800000]),
               (train_X[800001:900000,:],train_y[800001:900000]),
               (train_X[900001:1000000,:],train_y[900001:1000000]),
               (train_X[1000001:1100000,:],train_y[1000001:1100000]),
               (train_X[1100001:1200000,:],train_y[1100001:1200000]),
               (train_X[1200001:1300000,:],train_y[1200001:1300000]),
               (train_X[1300001:1400000,:],train_y[1300001:1400000]),
               (train_X[1400001:,:],train_y[1400001:])]

time: 3.7 ms


In [58]:
#for xs, ys in minibatches:
#    print(xs.shape, ys.shape)

(100000, 63574) (100000,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(99999, 63574) (99999,)
(43898, 63574) (43898,)
time: 1.34 ms


In [69]:
for xs, ys in minibatches:
    lr.partial_fit(xs, ys, classes=classes)

time: 15min 1s


In [ ]:
#clf = LogisticRegressionCV(cv=3,max_iter=200,n_jobs=-1)
#clf.fit(train_X,train_y)

#lr.fit(train_X,train_y)


In [70]:
lr.score(test_X, test_y)

0.9391121268785927

time: 2min 23s


In [ ]:
#clf.score(test_X,test_y)

In [71]:
lr.coef_ef_

array([[ 0.08101313, -0.07758269, -0.06293793, ..., -0.00172998,
         0.01258984,  0.00570538]])

time: 2.26 ms


In [72]:
lr.intercept_

array([-2.59587986])

time: 2.29 ms


In [73]:
from sklearn.metrics import classification_report

time: 497 µs


In [74]:
lr.loss_function_

time: 1.85 ms


In [75]:
lr.n_iter_

1

time: 1.71 ms


In [76]:
from sklearn.metrics import precision_recall_fscore_support

time: 461 µs


time: 772 ms


# Save Model

In [80]:
import pickle
# save the classifier
with open('my_dumped_classifier.pkl', 'wb') as fid:
    pickle.dump(lr, fid)    




time: 1.72 ms


In [82]:
# load it again
with open('my_dumped_classifier.pkl', 'rb') as fid:
    lr_loaded = pickle.load(fid)

time: 1.33 ms


In [83]:
lr_loaded.score(test_X, test_y)

0.9391121268785927

time: 2min 12s


Cool, still works

# Test data

In [90]:
test_orig = pd.read_csv("/home/sroberts/test.csv", engine='python')

time: 7.01 s


In [91]:
test_orig[:2]

,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwellian choices. He believes and has believed his entire career the exact opposite of what the position requires.
1,7000001,"I actually inspected the infrastructure on Grand Chief Stewart Philip's home Penticton First Nation in both 2010 and 2013. Exactly Zero projects that had been identified in previous inspection reports had been funded by the federal government, and the entire band was housed in ATCO trailers. Clearly the Harper Conservatives had already reduced the cash his band was sent to zero."


time: 5.55 ms


In [92]:
test_orig.fillna(0, inplace=True)
test_orig['clean'] = test_orig.comment_text.apply(lambda x: " ".join([words.strip(string.punctuation).lower()
for words in [word for word in x.split() if (word not in string.punctuation) & (word not in english_stopwords)]]))
test_orig["clean"] = test_orig["clean"].apply(lambda x: " ".join([lemmatizer.lemmatize(w) 
                                                                    for w in w_tokenizer.tokenize(x)]))
voc_counter = Counter(" ".join(test_orig.clean).split(" "))
voc_set = dict((k, v) for k, v in voc_counter.items() if v > 10)
voc_dict = {w:i for w,i in zip(voc_set, range(len(voc_set)))}
test_orig["clean"] = test_orig["clean"].apply(lambda x: x.split())

time: 17.8 s


In [93]:
#test_orig.loc[test_orig.target >= 0.5, "toxic"] = True
#test_orig.toxic.fillna(False,inplace=True)
X = np.zeros((len(test_orig), len(voc_set)), dtype = np.int8 )
for msg_idx, msg in enumerate(test_orig['clean']):
    for word in set(msg):
        if word in voc_dict:
            X[msg_idx][voc_dict[word]] = msg.count(word)
            
#y = np.where(test_orig.toxic == True, 1, 0)

time: 6.33 s


In [ ]:
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=.2,random_state=2)

In [97]:
lr.predict(X)

ValueError: X has 15337 features per sample; expecting 63574

time: 8.54 ms


In [96]:
X.shape

(97320, 15337)

time: 1.83 ms
